<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompt Engineering for Quality Decisions

Decision Intelligence applied in this module:  
* Optimizing prompts for logic, reasoning and decisions
* Examples of applying decision intelligence techniques in prompts and their output effect 
* Prompt techniques such as Chain of Thought to improve the quality of the decision reasoning and outcome 


In this module, the Semantic Kernel ability to chat completion experience will be used to optimize system prompts (personas) and instructive prompts. No new specific Semantic Kernel functionality will be introduced. This module will focus on introducing decision intelligence prompting techniques.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [2]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [3]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.56"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.6 Microsoft.SemanticKernel, 1.56.0

Using Azure OpenAI Service


### Step 2 - Chain of Thought Reasoning for Decision Intelligence 

> 📜 **_"The happiness of your life depends upon the quality of your thoughts."_**
>
> -- <cite>Marcus Aurelius (Roman emperor, philosopher)</cite>  

One of the most effective techniques to improve logic, reasoning and overall decision-making for most LLMs is to use "Chain of Thought" (CoT) techniques. The "Chain of Thought" approach helps by breaking down complex tasks into smaller steps, making it easier to solve problems without missing important details. It improves accuracy because each step is made clear to the LLM and helps avoid mistakes. This method also makes the thought process easier to understand and explain, allowing for easier corrections when something goes wrong. The approach is useful for handling difficult concepts and multi-step problems like math or logic, making things clearer and more manageable.

The name of the "Chain of Thought" technique has been recently popularized by Generative AI. However, this technique is not new nor is it specific only to Generative AI models. Breaking down complex tasks into simpler more approachable steps and organizing thoughts has been used by decision-makers, professional services organizations and management consulting companies for quite some time. Prior to Generative AI, the concepts have been known as: "structured thinking", "MECE framework" (Mutually Exclusive, Collectively Exhaustive), and "hypothesis-driven problem-solving".  

First, approach the problem with a simple Decision prompt technique. The system decisoin prompt will be set to basic & clear decision assistant persona instructions. While this decision prompt is simple, it is still quite effective to instruct the LLM model on how to approach the decision problem. 

In [5]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

### Problem Restatement

- The farmer needs to transport a wolf, a goat, and a cabbage across a river.
- The boat can only carry the farmer and one item at a time.
- If left alone together:
  - The wolf will eat the goat.
  - The goat will eat the cabbage.
- The goal: **Transport the goat across the river without it being eaten.**

### Step-by-Step Solution

#### 1. Analyze the Risks
- The goat is at risk of being eaten by the wolf if left alone with it.
- The goat will eat the cabbage if left alone with it.
- The farmer must ensure that the goat is never left alone with the wolf or the cabbage.

#### 2. Focus on the Goat
- The question specifically asks about transporting the goat across the river without it being eaten.

#### 3. Safe Sequence to Move the Goat

**Step 1:**  
- The farmer takes the goat across the river first.
- **Left side:** Wolf, Cabbage  
- **Right side:** Goat, Farmer

**Step 2:**  
- The farmer returns alone to the left side.
- **Left side:** Wolf, Cabbage, Farmer  
- **Right side:** Goat

**Result:**  
- The goat is now across the river, and at no point was it left alone with the wolf or the cabbage.

### Key Points

- The goat is never left alone with the wolf or the cabbage during this process.
- The farmer must always supervise the goat when it is with either the wolf or the cabbage.

### Summary Table

| Step | Left Side         | Right Side     | Action                       |
|------|-------------------|---------------|------------------------------|
| 1    | Wolf, Cabbage     | Goat, Farmer  | Farmer takes goat across     |
| 2    | Wolf, Cabbage, Farmer | Goat      | Farmer returns alone         |

### Conclusion

**To transport the goat across the river without it being eaten, the farmer should take the goat across first and return alone.** This ensures the goat is never left alone with the wolf or the cabbage.

---

Let's approach the same problem with a much more sophisticated "Chain of Thought" (CoT) system prompt to break the problem down and think about it more in depth. Notice that the prompt below is much more detailed in how the "decision intelligence assistant" should approach the problem, highlighting things it should or should not consider and the way it should arrive at the final answer. 

Chain of Thought (CoT) prompt instructions will vary depending on how the problem needs to be approached. It can include very specific decision framework instructions, systematic decision heuristics or management consulting best practices. In general, these detailed prompts result in the LLM providing more detail and usually better outcomes. However, there are a few drawbacks to overusing Chain of Thought (CoT):
* A very long and detailed Chain of Thought (CoT) can confuse the LLM model, especially if the LLM model is smaller (i.e. GPT-4o-mini or Phi-4 etc.)
* GenAI LLM models can hallucinate not just on output of answers. They can also hallucinate the Chain of Thought (CoT) they are describing in the final answer! This is a very sneaky way of potentially providing a confident answer paired with a confident approach (Chain of Thought) that the LLM may not even be using! 

In [ ]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemPromptChainOfThought = """
You are a Decision Intelligence assistant designed to think through problems step-by-step using Chain-of-Thought (CoT) prompting. 
Before providing any answer, you must: 

1) Understand the Problem: Carefully read and understand the user's question or request. 

2) Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 

3) Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Always aim to make your thought process transparent and logical.

4) Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 

5) Communicate the final decision using the Minto Pyramid Principle. 

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPromptChainOfThought);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var decisionResponseChainOfThought = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponseChainOfThought.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section


### Understanding the Problem

The farmer needs to transport a wolf, a goat, and a cabbage across a river. He can only take one item at a time in his boat. There are two critical constraints:
- If the wolf and goat are left alone together, the wolf will eat the goat.
- If the goat and cabbage are left alone together, the goat will eat the cabbage.

The question specifically asks: **How can the farmer transport the goat across the river without it being eaten?**

### Breaking Down the Reasoning Process

Let's clarify the problem:
- The farmer starts with all three items (wolf, goat, cabbage) on one side of the river.
- He wants to take the goat across first.
- We need to ensure that, at no point, the goat is left alone with the wolf or the cabbage on either side without the farmer present.

Let's analyze the possible outcomes step by step:

#### Step 1: Farmer takes the goat across first.
- Left side: Wolf, Cabbage
- Right side: Goat, Farmer

Is this safe?
- On the left side, wolf and cabbage are alone. The wolf does not eat the cabbage, so this is safe.
- On the right side, goat is with the farmer, so it's safe.

#### Step 2: Farmer returns alone to the left side.
- Left side: Wolf, Cabbage, Farmer
- Right side: Goat

Is this safe?
- On the left side, wolf and cabbage are alone with the farmer, so it's safe.
- On the right side, goat is alone. Is this safe? Yes, because the goat is not with the cabbage or wolf.

### Review the Thought Process

- The key risk is leaving the goat with either the wolf or the cabbage without the farmer.
- By taking the goat first, the wolf and cabbage are left together, which is safe.
- When the farmer returns alone, the goat is alone on the other side, which is also safe.

### Arriving at the Final Answer

#### Minto Pyramid Principle Communication

**Main Point:**  
The farmer can safely transport the goat across the river first without it being eaten.

**Supporting Points:**  
- Leaving the wolf and cabbage together is safe, as they do not harm each other.
- The goat is never left alone with either the wolf or the cabbage without the farmer present.

**Details:**  
1. The farmer takes the goat across the river and leaves it on the other side.
2. The farmer returns alone to the original side.

This sequence ensures the goat is not eaten during the initial crossing.

---

The Generative AI model has generated two different decision approaches to the same retirement decision question. An LLM can be used to decide which decision approach is of better quality. The evaluation could be done by another AI model or an AI model from different AI provider to reduce potential bias. 

After running the evaluation of the decision prompts below, notice the GenAI LLM model prefers the approach of the Chain of Thought (CoT) over the simple system prompt instruction. Furthermore, the AI model appreciates using the "Decision Communication" step with the applied Minto Pyramid.  

> 📝 **Note:** When executing the decision approach evaluation below, you may notice that sometimes Approach 1 is preferred. This happens as Approach 2 with Chain of Thought is considered verbose and not a clear as Approach 1 to the AI model.  In a real-world implementation, the prompt instruction will have this made very clear on what the criteria are. I specifically left that instruction out of the prompt to illustrate that even an AI model can vary it's preference!  

In [ ]:
var systemPromptEvaluateResponses = """
You are an assistant that is evaluating an approach response to a question.
You will be provided with an important decision as well as two proposed approaches.
The two approaches are labeled "Approach 1" and "Approach 2".

Compare the two approaches and evaluate them based on their: 
Logical structured reasoning of approach, detail of the approach to the decision and the quality the communication. 

Create a final score between 1 and 10 for each approach based on the evaluation criteria.
""";

var decisionEvaluationTemplateApproaches = $"""
Decision Scenario: 
{puzzlePrompt}
------------------------------------------------
Approach 1: 
{decisionResponse.Content} 
End of Approach 1
------------------------------------------------
Approach 2: 
{decisionResponseChainOfThought.Content} 
End of Approach 2
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistoryApproachEvaluation = new ChatHistory();
chatHistoryApproachEvaluation.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproachEvaluation.AddUserMessage(decisionEvaluationTemplateApproaches);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var evaluationResponseApproach1 = await chatCompletionService.GetChatMessageContentAsync(chatHistoryApproachEvaluation, openAIPromptExecutionSettings);
// Display the result of the chat completion request as Markdown
"### Decision Approaches Evaluation: ".DisplayAs("text/markdown");
evaluationResponseApproach1.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the output section

### Decision Approaches Evaluation: 

**Evaluation of Approaches**

---

### Approach 1

**Logical Structured Reasoning:**
- The approach is logically structured, starting with a restatement of the problem, followed by risk analysis, focus on the goat, and a clear step-by-step solution.
- The reasoning is methodical and ensures that all constraints are considered.
- The use of a summary table adds clarity and reinforces the steps.

**Detail of the Approach:**
- The approach details the risks, the sequence of actions, and explicitly states the state of each side after every move.
- It highlights why the goat is safe at each stage.
- The summary table is a helpful visual aid that makes the solution easy to follow.

**Quality of Communication:**
- The explanation is clear, concise, and well-organized.
- The use of headings and bullet points enhances readability.
- The conclusion succinctly summarizes the solution.

**Score: 10/10**

---

### Approach 2

**Logical Structured Reasoning:**
- The approach is logically sound and follows a clear step-by-step analysis.
- It restates the problem and constraints, then walks through the solution, checking for safety at each step.
- It includes a review of the thought process and uses the Minto Pyramid Principle for structured communication.

**Detail of the Approach:**
- The approach provides a detailed breakdown of each step, including the rationale for why each configuration is safe.
- It checks the safety of both sides after each move.
- However, it lacks a summary table or visual aid, which could have further clarified the steps.

**Quality of Communication:**
- The explanation is thorough and well-structured, with clear headings and logical flow.
- The use of the Minto Pyramid Principle is a nice touch, summarizing the main point and supporting details.
- The answer is slightly more verbose than necessary, but still clear.

**Score: 9/10**

---

## Final Scores

- **Approach 1:** 10/10
- **Approach 2:** 9/10

**Summary:**  
Both approaches are strong, but Approach 1 edges ahead due to its succinctness, clear risk analysis, and the inclusion of a summary table that visually reinforces the solution. Approach 2 is also well-reasoned and communicates clearly, but could benefit from a more concise summary or visual aid.

---

### Understanding Collective Intelligence for Decision Intelligence 

#### Collective Intelligence - Pooling Wisdom of Multiple Opinions 

> 📜 **_"The four conditions that characterize wise crowds: diversity of opinion, independence, decentralization, and aggregation."_**
>
> -- <cite>James Surowiecki (American journalist, author of The Wisdom of Crowds, and former columnist for The New Yorker)</cite> 

Imagine you’ve hurt your leg playing a long season of football games. You have another football season quickly approaching and you would like to understand how long will it take to heal properly. Additionally, you would like to understand the optimal medical treatment to ensure you are ready at the start of the next football season. You decide to visit three different doctors for their medical opinions. You don’t want to rely on just one doctor because each professional might notice something the others miss. Each medical specialist may decide to recommend a different approach based on their expertise or their own experience treating leg injuries. After you receive opinions from three different specialists, you think about their recommendations and then formulate a path forward (decision) for a treatment plan for your leg. The final treatment plan could be a result a variety of factors. For example, it could be simple and all three doctors could recommend 4-6 weeks rest. Clearly in that case there is a consensus with three different doctors on the treatment approach. What if the doctors diverge in their opinions? What if 2 of 3 doctors recommend rest and a third recommends an additional procedure on top of rest? What if all three doctors opinions totally diverge? Now you have to personally judge those doctor opinions collectively or potentially weight one doctor's opinion more significantly than others. This scenario illustrates a simplified version of collective intelligence, where pooling diverse expert opinions often leads to a more informed and balanced decision.

The idea of multiple experts arriving at a cohesive conclusion isn’t new. It has been demonstated that pooling the wisdom of multiple “opinions” can often outperform a single prediction. Those opinions don't have to be human experts. The opinions can be outcomes from a statistical model, a large survey, a machine learning model or Generative AI. You have probably have heard the terms: Wisdom of the Crowds, Collective Intelligence, Bootstrapping (technique in statistical analysis), Ensembling (Machine Learing) or Mixture of Experts (Generative AI). These are all similar techniques (with their own unique implementations) that derive of the core concept of "pooling wisdom of multiple opinions".  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SelfConsistency.png">

#### Examples of "Wisdom of the Crowds" at Scale

"Wisdom of the Crowds" can scale well beyond just a few doctor's opinions. In the examples below, note the number of samples collected for each situation is much greater than just a few doctors. In fact, the amount of samples collected can be in the hundreds and the Collective Intelligence power can hold true. 

**Francis Galton’s Ox-Weighing Experiment (1906)** 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-CollectiveIntelligence-OxWeight.png"> 

Sir Francis Galton, an English statistician, collected 787 guesses from fairgoers trying to estimate the weight of an ox. The average guess was 1,197 pounds, and the actual weight of the ox turned out to be 1,198 pounds. To Galton’s surprise, the average of these guesses (1,197 pounds) came remarkably close to the ox’s actual weight (1,198 pounds)—off by a single pound, or less than 0.1%. That level of accuracy was notable because the crowd was composed of a mixed audience: farmers with relevant experience, but also onlookers, tradespeople, and curious fairgoers with no particular expertise.

**Modern “Jelly Bean Jar” Contests**

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-CollectiveIntelligence-JellyBeans.png">

Many schools and charities run fundraisers where people pay to guess the number of jelly beans in a jar. It’s commonly observed that while individual guesses can be wildly off, the average of a sufficiently large group is typically within a small percentage (often under 5% error) of the true count. As an example, Michael Mauboussin ran an 2007 experiment where Mauboussin presented a jar of jelly beans to 73 Columbia Business School students. The students' guesses ranged from 250 to 4,100, with an average error of 62%. However, the group's average guess was 1,151, which represented only a 3% off the correct number. Only two students guessed better!

The cool part of the “Jelly Bean Jar” experiment is that it is very approachable to run yourself. In fact, you can do this with Generative AI as well. 

A short (5 minute) YouTube video illustrates running this experiment with human opinions:  
[![Jelly Bean Jar Experiment](https://img.youtube.com/vi/AuQdoAa2FUs/0.jpg)](https://www.youtube.com/watch?v=AuQdoAa2FUs)


**Netflix Recommendation Algorithm Million Dollar Prize**

In 2006, Netflix launched the million-dollar Netflix Prize competition, challenging participants to improve its movie recommendation algorithm (Cinematch) by at least 10 percent. Over three years, data scientists and researchers worldwide experimented with innovative approaches, culminating in 2009 when an ensemble team calling itself “BellKor’s Pragmatic Chaos” finally met the 10-percent threshold with 10.06%. This group had blended multiple models into a single “ensemble” algorithm that outperformed any single predictive approach. This demonstrated how synthesis of different predictions can be more powerful than one model alone. Although Netflix ultimately chose not to deploy the winning solution due to technical and privacy considerations, the competition is still regarded as a landmark moment in machine learning and collective problem-solving.  

More information can be found here: https://en.wikipedia.org/wiki/Netflix_Prize 

**“Ask the Audience” on Who Wants to Be a Millionaire?**

On the TV quiz show, contestants can use the “Ask the Audience” lifeline to poll the studio audience for the correct answer. Historical data shows that the audience collectively identifies the correct answer around 90% of the time, which is significantly more accurate than individual expert panels, or even the “Phone a Friend” lifeline.

**The U.S. Navy’s Hunt for the Missing Submarine Scorpion (1968)**

When the USS Scorpion, a nuclear-powered submarine, vanished in 1968, the Navy enlisted a broad range of specialists to harness their varied expertise and wisdom for a methodical search. By applying Bayesian statistical techniques to each person’s estimate of where the submarine might lie, they were able to synthesize these disparate perspectives into a single, remarkably accurate prediction. The team’s calculated guess put the submarine’s probable location just a few hundred yards from its actual resting place! This is a testament to the effectiveness of coalescing multiple expert viewpoints. This success with the USS Scorpion search later inspired more widespread use of Bayesian approaches for complex rescue and recovery operations, affirming the power of collective intelligence in high-stakes scenarios.

#### Statistics Explains why Collective Intelligence Works

It is important to understand that Collective Intelligence mechanisms are rooted in several statistics principles. This allows you to set up Collective Intelligence parameters appropriately for decision situations. Furthermore, this allows you to perform math (quantitative analysis) in the Decision Intelligence Execution step. The key statistical principles of Collective Intelligence:  
* **Average of Independent Predictions:** The average of many independent predictions, opinions or guesses will average itself out towards the true value. Each individual prediction can be assumed to have an error.  
Therefore the formula is:  Prediction = True Value + Error.  
As long as the errors are random, they will cancel each other out on average. 
* **Law of Large Numbers:** The more guesses you have, the more the random errors balance out, reducing the overall variance of the collective guess. Mathematically, if you have a large number of independent predictions, the average guess converges on the True Value. 
* **Central Limit Theorem:** The distribution of preductions will tend toward a bell curve (a normal distribution) around the Predicted True Value. When you take the average (mean) of a large sample from a sample distribution like this, it will often fall near the true population distribution. This happens more often in real-life than statistics courses lead on.  

Statistics explaining Collective Intelligence (Wisdom of the Crowds) is greatly simplfied and there are things to consider when attempting to replicate this experiment with real-world decision situations:  

* **Bias Matters:** Collective Intelligence works best when individual biases do not all tilt in the same direction. For example, if you only poll basketball players about height-related predictions, it may introduce a huge bias towards larger height in a single direction. 

* **Independence:** The group’s diversity and independence of guesses are critical. A group of people trained in the same way or over-influenced by each other’s guesses might show correlated errors, undermining the benefit of aggregation.

* **Meaningful Sample Size:** The sampling error shrinks as you collect more predictions (guesses). Assume you have 𝑁 amount of independent predictions. If each prediction's random error in guessing has a standard deviation of 𝜎, then the standard error of the mean guess drops by about a factor of $\sqrt{𝑁}$. What does this mean? The more samples you add, the more reduction of the error you will notice. Furthermore, you will notice a larger reduction in error with the first initial predictions than further predictions made. Saying it another way. For example, if you have a certain error from 10 samples and you want to reduce that prediction error by half you need a total of 40 samples. Conversely, if you have 100 samples and you want to reduce that prediction error by half you need a total of 400 samples! 

### Step 3 - Implementing Collective Intelligence with Generative AI 

Work in Progress - AI with Collective Intelligence 

In [16]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to plan retirement
var collectiveIntelligencePrompt = """
In this image there is a picture of a jar full to the top with jelly beans.
The jelly beans are of different colors and sizes, but they are all roughly the same size.
This image was taken from a video and includes other people and surroundings to provide a perspective of size of the jar.  

Esimtate the number of jelly beans in the jar. 
Think through the estimate clearly using all the available information in the image.

Provide only a single number as the answer.
""";

// Uri of a single image of a jar of jelly beans
Uri githubImageUri1 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans1.png");

// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.Add(
    new() {
        Role = AuthorRole.User,
        Items = [
            new TextContent { Text = collectiveIntelligencePrompt },
            new ImageContent { Uri = githubImageUri1 }
        ]
    });


// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    };

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

180

---

In [17]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to plan retirement
var collectiveIntelligencePrompt = """
You are provided with several images of a jar full to the top with jelly beans. 
The jelly beans are of different colors and sizes, but they are all roughly the same size.
The images were taken from a video and include other people and surroundings to provide a perspective of size of the jar.
The images provided are the same exact jar, but from different angles and distances. 

Esimtate the number of jelly beans in the jar. 
Think through the estimate clearly using all the available information in the image.

Provide only a single number as the answer.
""";

// Uri of a single image of a jar of jelly beans
Uri githubImageUri1 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans1.png");
Uri githubImageUri2 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans2.png");
Uri githubImageUri3 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans3.png");
Uri githubImageUri4 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans4.png");
Uri githubImageUri5 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/JellyBeans5.png");


// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.Add(
    new() {
        Role = AuthorRole.User,
        Items = [
            new TextContent { Text = collectiveIntelligencePrompt },
            new ImageContent { Uri = githubImageUri1 },
            new ImageContent { Uri = githubImageUri2 },
            new ImageContent { Uri = githubImageUri3 },
            new ImageContent { Uri = githubImageUri4 },
            new ImageContent { Uri = githubImageUri5 }
        ]
    });


// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    };

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

320

---